In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

In [2]:
df = pd.read_excel("output_file.xlsx",sheet_name="Sheet1")
df.head()

,no,comments,bank,date,platform,label,topik,Unnamed: 7
0,1,keren sekali transmart bintang melanda bintang...,Bank Mega,2023-05-07,Instagram,0,service,NaN
1,2,tidak pernah kabari menerapkan,Bank Mega,2023-05-07,Instagram,2,cc,NaN
2,3,keren bht,Bank Mega,2023-05-07,Instagram,0,service,NaN
3,4,transmart makssar masih kurang lengkap barangnya,Bank Mega,2023-05-07,Instagram,2,service,NaN
4,5,dan transmart keren oke wajah tersenyum dengan...,Bank Mega,2023-05-07,Instagram,0,service,NaN


In [8]:
import pandas as pd
excel_file_path = 'curr_train.xlsx'

sheets_dict = pd.read_excel(excel_file_path, sheet_name=None)
combined_df = pd.concat(sheets_dict.values(), ignore_index=True)

print(combined_df)

          no                                           comments       bank  \
0          1                         Keren banget Transmart 🤩🤩🤩  Bank Mega   
1          2                       gapernah di kabarin apply cc  Bank Mega   
2          3                                          Keren bht  Bank Mega   
3          4  Transmart di makssar masih kurang lengkap bara...  Bank Mega   
4          5                @bankmegaid & Transmart keren Oke 😍  Bank Mega   
...      ...                                                ...        ...   
58974  58975             Gimana bisa dapat lg mohon bantuan nya        BRI   
58975  58976  Kak untuk kak diary pkh bansos saya mau tnya s...        BRI   
58976  58977                                 pkh apa bnt ya bun        BRI   
58977  58978                   Mau pinjam tapi ga bisa  cara ny        BRI   
58978  58979                   Di coba kakak..offline juga bisa        BRI   

            date   platform Label (1,0,-1)  
0     2023-05-07  

In [3]:
file_path = 'id_stopwords.txt'
id_stopwords = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        id_stopwords.append(line.strip())

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('indonesian')
stop_words.extend(id_stopwords)
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.comments.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['keren', 'transmart', 'bintang', 'melanda', 'bintang', 'melanda', 'bintang', 'melanda']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmspa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 3), (1, 1), (2, 3), (3, 1)]


In [6]:
from pprint import pprint
# number of topics
num_topics = 3
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.035*"seriusbisa" + 0.032*"wajah" + 0.031*"beli" + 0.029*"mega" + '
  '0.027*"bank" + 0.023*"tersenyum" + 0.019*"hati" + 0.015*"semoga" + '
  '0.013*"ikutan" + 0.013*"mata"'),
 (1,
  '0.035*"beli" + 0.032*"wajah" + 0.029*"seriusbisa" + 0.021*"mata" + '
  '0.018*"mega" + 0.017*"tersenyum" + 0.016*"bank" + 0.012*"hati" + '
  '0.009*"tangan" + 0.008*"air"'),
 (2,
  '0.067*"hati" + 0.038*"seriusbisa" + 0.034*"beli" + 0.028*"wajah" + '
  '0.026*"merah" + 0.025*"mata" + 0.019*"tersenyum" + 0.010*"bank" + '
  '0.010*"kartu" + 0.009*"kredit"')]


In [7]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

FileNotFoundError: [Errno 2] No such file or directory: '/results/ldavis_prepared_3'